In [1]:
import pandas as pd
import numpy as np

In [2]:
data=pd.read_csv('feature_engineering0212.csv')

/Applications/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data.head()

,Unnamed: 0,user_id,city_id,sex,age,occupation,invite_id,invite_code,login_platform,last_login,...,06/11/2016count,06/11/2016send_type_mode,06/11/2016couponworth_sum,06/11/2016ticketfee_sum,06/11/2016couponfee_sum,06/12/2016count,06/12/2016send_type_mode,06/12/2016couponworth_sum,06/12/2016ticketfee_sum,06/12/2016couponfee_sum
0,0,100002020,0,1,80,互联网,0,9828.0,Android,2016-07-04 17:53:27.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13,100002033,0,0,NaN,NaN,0,6126.0,iOS,2017-05-12 00:42:20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15,100002035,0,0,NaN,NaN,0,3227.0,iOS,2016-01-13 17:29:46.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,16,100002036,0,0,NaN,NaN,0,3844.0,iOS,2017-04-19 12:59:09.0,...,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0
4,20,100002040,0,2,90,上班族,0,4316.0,Android,2017-02-13 16:56:55.0,...,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0


In [4]:
len(data[data['label']==1])/len(data)

0.29499882723932136

In [5]:
columnlist=data.columns.tolist()

In [6]:
len(columnlist)

363

In [7]:
data_sd=data.filter(like='count', axis=1).fillna(0)

In [8]:
data_sd['combined']= data_sd.values.tolist()

In [9]:
import statistics
statistics.stdev(data_sd['combined'].iloc[2])

0.3916837360273247

In [10]:
data_sd['order_sd']=data_sd['combined'].apply(lambda x: statistics.stdev(x))

In [11]:
final=data[['user_id']]
final['order_sd']=data_sd['order_sd']

/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [12]:
data_sd['active_days']=data_sd['combined'].apply(lambda x: len(x)-x.count(0))

In [13]:
final['active_days']=data_sd['active_days']

/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [14]:
data_sd['combined_to1']=data_sd['combined'].apply(lambda x: [1 if i > 0 else 0 for i in x])

In [15]:
data_sd['combined_to1'].iloc[1].index(1)

1

In [16]:
data_sd['findindex']=data_sd['combined_to1'].apply(lambda x:[i for i, j in enumerate(x) if j == 1])
data_sd['findindex'].iloc[1]

[1, 2, 3, 4, 7, 8, 10, 14, 15, 16, 21, 22, 25, 36, 40, 42, 44, 51]

In [17]:
data_sd['lastgap']=data_sd['findindex'].apply(lambda x: np.nan if len(x)<=2 else x[-2:][1]-x[-2:][0] )

In [18]:
final['lastgap']=data_sd['lastgap']

/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [19]:
temp=[1,2,5]

In [20]:
[j-i for i, j in zip(temp[:-1], temp[1:])]

[1, 3]

In [21]:
data_sd['gaps']=data_sd['findindex'].apply(lambda x: [j-i for i, j in zip(x[:-1], x[1:])] )
data_sd['gaps'].iloc[1]

[1, 1, 1, 3, 1, 2, 4, 1, 1, 5, 1, 3, 11, 4, 2, 2, 7]

In [22]:
from statistics import mean

In [23]:
mean(data_sd['gaps'].iloc[1])

2.9411764705882355

In [24]:
data_sd['avg_gap']=data_sd['gaps'].apply(lambda x: mean(x) if len(x)>=1 else np.nan )

In [25]:
data_sd['min_gap']=data_sd['gaps'].apply(lambda x: min(x) if len(x)>=1 else np.nan )
data_sd['max_gap']=data_sd['gaps'].apply(lambda x: max(x) if len(x)>=1 else np.nan )

In [26]:
final['avg_gap']=data_sd['avg_gap']
final['min_gap']=data_sd['min_gap']
final['max_gap']=data_sd['max_gap']

/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

In [27]:
data_sd['total_orders']=data_sd['combined'].apply(lambda x: sum(x) if len(x)>=1 else np.nan )

In [28]:
final['total_orders']=data_sd['total_orders']

/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [29]:
weeks=data.filter(like='count', axis=1).fillna(0)

len(weeks.columns.tolist())

week1=weeks.columns.tolist()[0:10]
week2=weeks.columns.tolist()[10:20]
week3=weeks.columns.tolist()[20:30]
week4=weeks.columns.tolist()[30:40]
week5=weeks.columns.tolist()[40:50]
week6=weeks.columns.tolist()[50:60]
week7=weeks.columns.tolist()[60:70]

In [30]:
data_week1=data[week1].fillna(0)
data_week1['combined']= data_week1.values.tolist()

data_week1['w1_total_orders']=data_week1['combined'].apply(lambda x: sum(x))

data_week2=data[week2].fillna(0)
data_week2['combined']= data_week2.values.tolist()

data_week2['w2_total_orders']=data_week2['combined'].apply(lambda x: sum(x))

data_week3=data[week3].fillna(0)
data_week3['combined']= data_week3.values.tolist()

data_week3['w3_total_orders']=data_week3['combined'].apply(lambda x: sum(x))

data_week4=data[week4].fillna(0)
data_week4['combined']= data_week4.values.tolist()

data_week4['w4_total_orders']=data_week4['combined'].apply(lambda x: sum(x))

data_week5=data[week5].fillna(0)
data_week5['combined']= data_week5.values.tolist()

data_week5['w5_total_orders']=data_week5['combined'].apply(lambda x: sum(x))

data_week6=data[week6].fillna(0)
data_week6['combined']= data_week6.values.tolist()

data_week6['w6_total_orders']=data_week6['combined'].apply(lambda x: sum(x))

data_week7=data[week7].fillna(0)
data_week7['combined']= data_week7.values.tolist()

data_week7['w7_total_orders']=data_week7['combined'].apply(lambda x: sum(x))

final['w1_total_orders']=data_week1['w1_total_orders']
final['w2_total_orders']=data_week2['w2_total_orders']
final['w3_total_orders']=data_week3['w3_total_orders']
final['w4_total_orders']=data_week4['w4_total_orders']
final['w5_total_orders']=data_week5['w5_total_orders']
final['w6_total_orders']=data_week6['w6_total_orders']
final['w7_total_orders']=data_week7['w7_total_orders']


/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/

In [31]:
data_coupon=data.filter(like='couponworth_sum', axis=1).fillna(0)

In [32]:
data_coupon.head()

,04/04/2016couponworth_sum,04/05/2016couponworth_sum,04/06/2016couponworth_sum,04/07/2016couponworth_sum,04/08/2016couponworth_sum,04/09/2016couponworth_sum,04/10/2016couponworth_sum,04/11/2016couponworth_sum,04/12/2016couponworth_sum,04/13/2016couponworth_sum,...,06/03/2016couponworth_sum,06/04/2016couponworth_sum,06/05/2016couponworth_sum,06/06/2016couponworth_sum,06/07/2016couponworth_sum,06/08/2016couponworth_sum,06/09/2016couponworth_sum,06/10/2016couponworth_sum,06/11/2016couponworth_sum,06/12/2016couponworth_sum
0,0.0,22.0,3.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,8.0,5.0,0.0,0.0,0.0,17.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
2,0.0,16.5,0.0,0.0,0.0,0.0,4.0,0.0,20.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,9.6,0.0,8.96,0.0,0.0,0.0,0.0,0.0


In [33]:
data_coupon['combined']= data_coupon.values.tolist()

In [34]:
data_coupon['total_couponworth']=data_coupon['combined'].apply(lambda x: sum(x)  )

In [35]:
final['total_couponworth']=data_coupon['total_couponworth']

/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [36]:
weeks=data.filter(like='couponworth', axis=1).fillna(0)

len(weeks.columns.tolist())

week1=weeks.columns.tolist()[0:10]
week2=weeks.columns.tolist()[10:20]
week3=weeks.columns.tolist()[20:30]
week4=weeks.columns.tolist()[30:40]
week5=weeks.columns.tolist()[40:50]
week6=weeks.columns.tolist()[50:60]
week7=weeks.columns.tolist()[60:70]

In [37]:
data_week1=data[week1].fillna(0)
data_week1['combined']= data_week1.values.tolist()

data_week1['w1_total_couponworth']=data_week1['combined'].apply(lambda x: sum(x))

data_week2=data[week2].fillna(0)
data_week2['combined']= data_week2.values.tolist()

data_week2['w2_total_couponworth']=data_week2['combined'].apply(lambda x: sum(x))

data_week3=data[week3].fillna(0)
data_week3['combined']= data_week3.values.tolist()

data_week3['w3_total_couponworth']=data_week3['combined'].apply(lambda x: sum(x))

data_week4=data[week4].fillna(0)
data_week4['combined']= data_week4.values.tolist()

data_week4['w4_total_couponworth']=data_week4['combined'].apply(lambda x: sum(x))

data_week5=data[week5].fillna(0)
data_week5['combined']= data_week5.values.tolist()

data_week5['w5_total_couponworth']=data_week5['combined'].apply(lambda x: sum(x))

data_week6=data[week6].fillna(0)
data_week6['combined']= data_week6.values.tolist()

data_week6['w6_total_couponworth']=data_week6['combined'].apply(lambda x: sum(x))

data_week7=data[week7].fillna(0)
data_week7['combined']= data_week7.values.tolist()

data_week7['w7_total_couponworth']=data_week7['combined'].apply(lambda x: sum(x))

final['w1_total_couponworth']=data_week1['w1_total_couponworth']
final['w2_total_couponworth']=data_week2['w2_total_couponworth']
final['w3_total_couponworth']=data_week3['w3_total_couponworth']
final['w4_total_couponworth']=data_week4['w4_total_couponworth']
final['w5_total_couponworth']=data_week5['w5_total_couponworth']
final['w6_total_couponworth']=data_week6['w6_total_couponworth']
final['w7_total_couponworth']=data_week7['w7_total_couponworth']



/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/

In [38]:
final['label']=data['label']

/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [39]:
final.to_csv('Mengdan_features.csv',index=False)

In [ ]:
final=pd.read_csv('Mengdan_features.csv')

In [40]:
final.shape

(191855, 24)

In [41]:
final.head()

,user_id,order_sd,active_days,lastgap,avg_gap,min_gap,max_gap,total_orders,w1_total_orders,w2_total_orders,...,w7_total_orders,total_couponworth,w1_total_couponworth,w2_total_couponworth,w3_total_couponworth,w4_total_couponworth,w5_total_couponworth,w6_total_couponworth,w7_total_couponworth,label
0,100002020,0.663138,10,3.0,4.888889,1.0,12.0,16.0,4.0,2.0,...,0.0,126.00,45.0,17.0,15.0,33.0,16.0,0.00,0.00,0.0
1,100002033,0.602960,18,7.0,2.941176,1.0,11.0,22.0,8.0,5.0,...,0.0,101.00,30.0,19.0,8.0,15.0,16.0,13.00,0.00,0.0
2,100002035,0.391684,13,5.0,4.500000,2.0,12.0,13.0,3.0,2.0,...,0.0,140.50,40.5,25.0,23.0,0.0,27.0,25.00,0.00,0.0
3,100002036,0.337142,9,5.0,8.625000,1.0,53.0,9.0,2.0,0.0,...,3.0,5.00,0.0,0.0,0.0,0.0,0.0,5.00,0.00,1.0
4,100002040,0.535490,12,5.0,3.454545,1.0,6.0,15.0,0.0,0.0,...,5.0,45.48,0.0,0.0,0.0,3.0,14.0,9.92,18.56,1.0


In [42]:
data_xinwen=pd.read_csv('feature_engineering_xwpart.csv')

In [43]:
len(data_xinwen)

191865

In [44]:
len(final)

191855

In [63]:
set(data_xinwen.user_id)-set(final.user_id)

set()

In [67]:
set(data_xinwen.user_id)==set(final.user_id)

True

In [65]:
len(data_xinwen.user_id.unique())

191855

In [66]:
len(final.user_id.unique())

191855

In [64]:
set(final.user_id)-set(data.user_id)

set()

In [45]:
data_xinwen.head()

,user_id,first_order,last_order,accountLength,distance_min,distance_max,distance_mean,overall_time_mean,update2create_time_mean,send2update_time_mean,...,update2create_time_min,send2update_time_min,overall_time_max,update2create_time_max,send2update_time_max,intime_rate,last_coupon_fee,last_send_type,last_distance,order_nums
0,100002020,68,24,626,0.0,6011.0,946.562500,4007.250000,2887.500000,2463.500000,...,1182.0,2095.0,9404.0,7309.0,2751.0,0.937500,9.0,1,56.0,8.0
1,100002033,68,18,626,524.0,2666.0,1646.500000,7422.545455,4852.619048,2737.333333,...,36.0,-132.0,63013.0,63145.0,8206.0,1.000000,7.0,1,2099.0,10.0
2,100002035,68,14,625,1069.0,1764.0,1313.000000,8809.923077,6259.384615,2550.538462,...,1664.0,1496.0,21959.0,18588.0,3524.0,1.000000,10.0,1,1423.0,7.0
3,100002036,69,0,625,1016.0,2237.0,1647.888889,8093.777778,5610.777778,2483.000000,...,162.0,526.0,17242.0,15155.0,4031.0,1.000000,0.0,1,1876.0,5.0
4,100002040,38,0,625,690.0,4513.0,1707.800000,4369.533333,2197.785714,2199.928571,...,932.0,283.0,7915.0,5828.0,3324.0,0.933333,0.0,1,2140.0,2.0


In [46]:
final2=pd.merge(final,data_xinwen,how='left',on='user_id')

In [47]:
len(final2.columns)

44

In [48]:
final2.to_csv('all_data.csv',index=False)

In [49]:
len(final2)

191865

In [50]:
data_count=data.filter(like='count', axis=1).fillna(0)

In [51]:
data_count['user_id']=data['user_id']

In [52]:
final3=pd.merge(final2,data_count,how='left',on='user_id')

In [53]:
final3.to_csv('all_data_add_daily_feature.csv',index=False)

In [54]:
final2.head()

,user_id,order_sd,active_days,lastgap,avg_gap,min_gap,max_gap,total_orders,w1_total_orders,w2_total_orders,...,update2create_time_min,send2update_time_min,overall_time_max,update2create_time_max,send2update_time_max,intime_rate,last_coupon_fee,last_send_type,last_distance,order_nums
0,100002020,0.663138,10,3.0,4.888889,1.0,12.0,16.0,4.0,2.0,...,1182.0,2095.0,9404.0,7309.0,2751.0,0.937500,9.0,1,56.0,8.0
1,100002033,0.602960,18,7.0,2.941176,1.0,11.0,22.0,8.0,5.0,...,36.0,-132.0,63013.0,63145.0,8206.0,1.000000,7.0,1,2099.0,10.0
2,100002035,0.391684,13,5.0,4.500000,2.0,12.0,13.0,3.0,2.0,...,1664.0,1496.0,21959.0,18588.0,3524.0,1.000000,10.0,1,1423.0,7.0
3,100002036,0.337142,9,5.0,8.625000,1.0,53.0,9.0,2.0,0.0,...,162.0,526.0,17242.0,15155.0,4031.0,1.000000,0.0,1,1876.0,5.0
4,100002040,0.535490,12,5.0,3.454545,1.0,6.0,15.0,0.0,0.0,...,932.0,283.0,7915.0,5828.0,3324.0,0.933333,0.0,1,2140.0,2.0


In [55]:
data_user=pd.read_csv('userInfo.csv')

/Applications/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [56]:
data_user.columns

Index(['Unnamed: 0', 'user_id', 'city_id', 'sex', 'age', 'occupation',
       'invite_id', 'invite_code', 'login_platform', 'last_login',
       'app_version', 'create_time'],
      dtype='object')

In [57]:
data_user=data_user[['user_id', 'city_id', 'sex', 'age', 'occupation',
       'invite_id', 'invite_code', 'login_platform', 
       'app_version']]

In [58]:
final2=pd.merge(final2,data_user,how='left',on='user_id')

In [59]:
len(final2)

191865

In [60]:
final2.to_csv('all_data.csv',index=False)

In [61]:
final3=pd.merge(final3,data_user,how='left',on='user_id')

In [62]:
final3.to_csv('all_data_add_daily_feature.csv',index=False)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Applications/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2862, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-62-3aa4fba0c2d0>", line 1, in <module>
    final3.to_csv('all_data_add_daily_feature.csv',index=False)
  File "/Applications/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py", line 1745, in to_csv
    formatter.save()
  File "/Applications/anaconda3/lib/python3.6/site-packages/pandas/io/formats/csvs.py", line 171, in save
    self._save()
  File "/Applications/anaconda3/lib/python3.6/site-packages/pandas/io/formats/csvs.py", line 286, in _save
    self._save_chunk(start_i, end_i)
  File "/Applications/anaconda3/lib/python3.6/site-packages/pandas/io/formats/csvs.py", line 310, in _save_chunk
    quoting=self.quoting)
  File "/Applications/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/base.py", line 2432, in to_native_types
    return value

KeyboardInterrupt: 